In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# 1. Load enriched cities
cities = pd.read_csv('../data/cities_with_econ.csv')

# We only need a DataFrame here, no spatial join
# 2. Load your store locations with pre-filled city_name
stores = pd.read_csv('../data/store_locations.csv')
# Trim whitespace if any:
stores['city_name'] = stores['city_name'].str.strip()

# 3. Count stores per city directly
store_counts = (
    stores.groupby('city_name')
          .size()
          .rename('store_count')
          .reset_index()
)

# 4. Merge counts back into our cities DataFrame
cities = cities.merge(
    store_counts,
    how='left',
    left_on='name',
    right_on='city_name'
)
cities['store_count'] = cities['store_count'].fillna(0).astype(int)

# Preview
cities[['name','countrycode','store_count']].sort_values('store_count', ascending=False).head()


,name,countrycode,store_count
26352,Chitungwiza,ZW,0
26336,Karoi,ZW,0
26335,Kwekwe,ZW,0
26334,Marondera,ZW,0
26333,Masvingo,ZW,0


In [2]:
# Normalize store counts
cities['comp_intensity'] = cities['store_count'] / cities['store_count'].max()

# Build final expansion score (high econ_viability & low competition)
cities['expansion_score'] = cities['econ_viability'] * (1 - cities['comp_intensity'])

# Show your top 10 cities
top10 = cities.sort_values('expansion_score', ascending=False).head(10)
top10[['name','countrycode','econ_viability','store_count','comp_intensity','expansion_score']]


,name,countrycode,econ_viability,store_count,comp_intensity,expansion_score
0,les Escaldes,AD,0.107241,0,NaN,NaN
1,Andorra la Vella,AD,0.110053,0,NaN,NaN
2,Umm Al Quwain City,AE,0.144113,0,NaN,NaN
3,Ras Al Khaimah City,AE,0.166608,0,NaN,NaN
4,Zayed City,AE,0.144265,0,NaN,NaN
5,Khawr Fakkān,AE,0.138459,0,NaN,NaN
6,Dubai,AE,0.196493,0,NaN,NaN
7,Dibba Al-Fujairah,AE,0.134487,0,NaN,NaN
8,Dibba Al-Hisn,AE,0.132817,0,NaN,NaN
9,Sharjah,AE,0.183398,0,NaN,NaN


In [3]:
cities.to_csv('../data/cities_final_ranked.csv', index=False)
# 5. Save the final cities DataFrame with expansion scores